In [126]:
from ngsolve import *
from netgen.geom2d import SplineGeometry
from ngsolve.webgui import Draw
import copy

setup geometry

In [127]:
geo = SplineGeometry()
geo.AddRectangle( (-3,-2), (3, 2), bcs = ("top", "out", "bot", "in"))
geo.AddCircle ( (0, 0), r=0.5, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.2))
mesh.Curve(3);
mesh_orig = copy.deepcopy(mesh)
#Draw(mesh)

setup FEM space

In [128]:
# viscosity
nu = 0.01
# Order of spaces
k = 2
# H1 vs VectorH1 -> vector field?!
V = H1(mesh,order=k, dirichlet="top|bot|cyl|in|out")
Q = H1(mesh,order=k-1)
FES = FESpace([V,V,Q]) # X = [V,V,Q] (without VectorH1)

setup bilinear form
velocityfield u and pressurefield p

In [129]:
ux,uy,p = FES.TrialFunction()
vx,vy,q = FES.TestFunction()

div_u = grad(ux)[0]+grad(uy)[1]
div_v = grad(vx)[0]+grad(vy)[1]


# (InnerProduct(grad(u),grad(v))+div(u)*q+div(v)*p)*dx
a = BilinearForm(FES)
a += (grad(ux)*grad(vx)+grad(uy)*grad(vy) + div_u*q + div_v*p) * dx
a.Assemble()

setup boundary conditions

In [130]:
gfu = GridFunction(FES)
#'gfp = GridFunction(X)
# setup flow condition
uinf = 0.001
uin = CoefficientFunction((uinf))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("in|top|bot|out"))

velocity = CoefficientFunction(gfu.components[0])
scene_state = Draw(velocity, mesh, "vel")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

solve equation

In [131]:
def solveStokes():
    res = gfu.vec.CreateVector()
    res.data = -a.mat * gfu.vec
    inv = a.mat.Inverse(FES.FreeDofs())
    gfu.vec.data += inv * res
    scene_state.Redraw()
solveStokes()

### creation of gfset
function that will perturb our mesh

In [132]:
# Test and trial functions for shape derivate -> do we even need this?
VEC = H1(mesh, order=2, dim=2, dirichlet="top|bot|in|out")
PHI, X = VEC.TnT()
# gfset denotes the deformation of the original domain and will be updated during the shape optimization
gfset = GridFunction(VEC)
gfset.Set((0,0))
mesh.SetDeformation(gfset)
#sceneSet = Draw(gfset,mesh,"gfset")
SetVisualization (deformation=True)

# deformation calculation?!
gfX = GridFunction(VEC)

Drag/ "dissipated energy" [should be working]

$J(\Omega ) = \frac{1}{2} \int_\Omega Du : Du dx$ 

In [133]:
solveStokes()
vel = gfu.components[0] #gfu.vec.data
drag = Integrate(0.5*InnerProduct(Grad(vel),Grad(vel)),mesh)
print(drag)

# 1.0997749905576028e-05 - r=0.5
# 0.00043084429187575745 - r=1.5

9.09406040897443e-06


In [134]:
def calc_drag(gfu):
    ux = gfu.components[0]
    uy = gfu.components[1]
    return 0.5*(grad(ux)*grad(ux)+grad(uy)*grad(uy))*dx

### get surface area of mesh [done]
(without ball), should stay constant

In [135]:
surf_t = CoefficientFunction(1)
surf_0 = Integrate(surf_t,mesh)

def calc_surf():
    return surf_t*dx-surf_0

def calc_surf_inv():
    return 1/(surf_t*dx)

def calc_surf_diff():
    return (surf_0 - surf_t*dx)**2

1/Integrate(surf_t,mesh)

0.04307635342978572

### try for barycenter <br>
$vol(\Omega) = \int_{\Omega} 1 \,dx \in \mathbb{R}$ <br>
von Hand abeleiten

$bc^\Omega = \frac{1}{vol(\Omega)}\int_{\Omega}x\,dx \in \mathbb{R}^d$


In [136]:
#real calculation:
print((1/Integrate(1,mesh))*Integrate(x,mesh))
print((1/Integrate(1,mesh))*Integrate(y,mesh))

-1.4945112310824668e-17
6.501123855208731e-18


In [137]:
bc_t = CoefficientFunction(x)
bc_0 = 1/surf_0*Integrate(bc_t,mesh)
print(bc_0)
to = 1/CoefficientFunction(1)*CoefficientFunction(x)
Integrate(to,mesh)

-1.4945112310824668e-17


-3.469446951953614e-16

In [138]:
bc_t = CoefficientFunction(x)
bc_0 = 1/surf_0*Integrate(bc_t,mesh)

def calc_bc():
    return (calc_surf_inv()*(bc_t*dx))# - bc_0
    #return bc_t*dx
    
def calc_bc_test():
    return 1/CoefficientFunction(1)*dx*CoefficientFunction(x+y)*dx # does not work

In [139]:
def Cost(gfu):
    standard = CoefficientFunction(0)*dx
    c = calc_drag(gfu)
    return c

In [140]:
n = specialcf.normal(2)
def DiffCost(gfu):
    #return 100*div(gfset)**2*dx
    return (X*n)**2*ds(definedon="cyl")

## Adjoint equation try?
maybe we dont even need this??

* J hängt von u ab, u von omega
* dJ nach u -> "in Vektorfeldrichtung" -> für jedes Vektorfeld ableiten
* stattdessen adjoint -> 1x u und 1x omega
* adjoint
* Lagrangian du -> adjoint

In [141]:
gfp = GridFunction(FES)
r, w = FES.TnT()
fadjoint = LinearForm(FES)
fadjoint += -1*(Cost(gfu)).Diff(gfu,w)
# add by hand derived formula for constant volume:
fadjoint += -1*(DiffCost(gfu))

In [142]:
def SolveAdjointEquation():
    rhs = gfp.vec.CreateVector()
    rhs.data = fadjoint.vec - a.mat.T * gfp.vec
    update = gfp.vec.CreateVector()
    update.data = a.mat.Inverse(FES.FreeDofs()).T * rhs
    gfp.vec.data += update

In [143]:
fadjoint.Assemble()
SolveAdjointEquation()
adjoint_sol = CoefficientFunction(gfp.components[0])
Draw (adjoint_sol, mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

### Shape derivative $\mathrm{d}J(\Omega)(\mathbf{X}) = \int_{\Omega} \mathbf{S}_1:D \mathbf{X} dx$
$S_1 = \biggl(\frac{1}{2}Du : Du - p div(u) \biggr) I_2 + Du^T p - Du^T Du.$ <br>
$\mathrm{u}$ and $p$ are the solutions to the stokesflow problem<br>
we use u = gfu[0], p = gfu[1]

# Auto shapediff try

In [144]:
def Equation(u,v):
    ux = u.components[0]
    uy = u.components[1]
    p = u.components[2]
    vx = v.components[0]
    vy = v.components[1]
    q = v.components[2]
    div_u = grad(ux)[0]+grad(uy)[1]
    div_v = grad(vx)[0]+grad(vy)[1]
    return (grad(ux)*grad(vx)+grad(uy)*grad(vy) + div_u*q + div_v*p) * dx
    #return (InnerProduct(grad(u.components[0]),grad(v.components[0]))+div(u.components[0])*v.components[1]+div(v.components[0])*u.components[1])*dx # stokes
    #return (0.5*(InnerProduct(grad(u.components[0]),grad(u.components[0]))-v*div(u))*Id(2)+grad(u).trans*v-grad(u).trans*grad(u))*dx # shape derivative
    
    
    
Lagrangian = Equation(gfu,gfp) + DiffCost(gfu)  + Cost(gfu)

In [145]:
dJOmega = LinearForm(VEC)
dJOmega += Lagrangian.DiffShape(PSI)
#dJOmega += Equation(PHI,PSI)
#dJOmega += DiffCost

b = BilinearForm(VEC)
b += InnerProduct(grad(PSI),grad(PHI))*dx + InnerProduct(PSI,PHI)*dx

# gfX = GridFunction(VEC)

## this basically just copy paste

In [146]:
def SolveDeformationEquation():
    rhs = gfX.vec.CreateVector()
    rhs.data = dJOmega.vec - b.mat * gfX.vec
    update = gfX.vec.CreateVector()
    update.data = b.mat.Inverse(VEC.FreeDofs()) * rhs
    gfX.vec.data += update

In [147]:
b.Assemble()
dJOmega.Assemble()
SolveDeformationEquation()
Draw(-gfX, mesh, "-gfX")

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

BaseWebGuiScene

In [148]:
print('Cost at initial design', Integrate (Cost(gfu), mesh))
gfset.Set((0,0))
scale = 0.5 / Norm(gfX.vec)
gfset.vec.data -= scale * gfX.vec

Cost at initial design 1.0997830056224484e-05


In [149]:
test_scene = Draw(gfset)
mesh.SetDeformation(gfset)
test_scene.Redraw()

a.Assemble()
fadjoint.Assemble()
solveStokes()
print('Cost at new design', Integrate (Cost(gfu), mesh))

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

Cost at new design 8.893692578283055e-06


In [150]:
#reset the design
gfset.Set((0,0))
mesh.SetDeformation(gfset)

#check if initial value of cost function 0.000486578350214552 is recovered
a.Assemble()
fadjoint.Assemble()
solveStokes()
print('Cost at new design', Integrate (Cost(gfu), mesh))

Cost at new design 1.0997830056224496e-05


# general idea: of iteration

* solve solution
* deform
* calc cost after deform
* look how cost changed -> armijo rule
* abstieg: -gradient²

In [151]:
scene = Draw(gfset)

WebGuiWidget(value={'ngsolve_version': '6.2.2201', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'draw_vol': Fals…

In [152]:
gfset.Set((0,0))
mesh.SetDeformation(gfset)
scene.Redraw()
a.Assemble()
fadjoint.Assemble()
solveStokes()

LineSearch = False

iter_max = 600
Jold = Integrate(Cost(gfu)+DiffCost(gfu), mesh)
converged = False

# try parts of loop
mesh.SetDeformation(gfset)
scene.Redraw()

print('cost at iteration', k, ': ', Jold)
    
# input("Press enter to start optimization")
for k in range(iter_max):
    mesh.SetDeformation(gfset)
    scene.Redraw()
    scene_state.Redraw()
    
    print('cost at iteration', k, ': ', Jold)
    a.Assemble()
    solveStokes()
    
    fadjoint.Assemble()
    SolveAdjointEquation()
    
    b.Assemble()
    dJOmega.Assemble()
    SolveDeformationEquation()

    #scale = 0.01 / Norm(gfX.vec)
    scale = 0.1 / Norm(gfX.vec)
    gfsetOld = gfset
    gfset.vec.data -= scale * gfX.vec
    Jnew = Integrate(Cost(gfu)+DiffCost(gfu), mesh)
    
    if LineSearch:
        while Jnew > Jold and scale > 1e-12:
            #input('a')
            scale = scale / 2
            print("scale = ", scale)
            if scale <= 1e-12:
                converged = True
                break

            gfset.vec.data = gfsetOld.vec - scale * gfX.vec
            mesh.SetDeformation(gfset)
            
            a.Assemble()
            solveStokes()
            Jnew = Integrate(Cost(gfu)+DiffCost(gfu), mesh)
    
    if converged==True:
        print("No more descent can be found")
        break
    Jold = Jnew

    Redraw(blocking=True)

NgException: cannot evaluate ProxyFunction without userdata